# PID controller example

In the following noteook I use [Cart Pole](https://www.gymlibrary.dev/environments/classic_control/cart_pole/) Gymnasium enviromnent to test PID controller mechanics

PID controller details on [Wikipedia](https://en.wikipedia.org/wiki/Proportional%E2%80%93integral%E2%80%93derivative_controller)

The classical PID formula includes 3 components: error, error's derivative, error's integral. Let $e$ be an error of the control loop and $u$ be a control variable

$$
u = e \cdot K_p + \frac{de}{dt} K_d + \int e \cdot dt * K_i
$$

Still, everything does not fill to Cart Pole env out of the box


**Problem 1**. Cart Pole has two objectives: keep the pole in the vertical position and keep it centered.

Tweak 1. Use a composite error

$$
e = e_{angle} * 0.1 * e_{position}
$$

**Problem 2** Actions are binary (0 or 1)

Tweak 2. Binarize PID output

$$
Action = sign(u)
$$

**Results**

1. I found coeffs manually. It required a really short amount of time
2. Integral component appeared to be useless in this environment

![Cart Pole GIF should be here](https://www.gymlibrary.dev/_images/cart_pole.gif)

In [1]:
import gymnasium as gym

In [2]:
class PIDAgent:
    def __init__(self, kp, ki, kd):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.integral_error = 0
        self.prev_error = 0
    
    def get_action(self, observation):
        error = observation[2] + 0.1 * observation[0]
        diff_error = error - self.prev_error
        self.integral_error += error
        p = error * self.kp + self.integral_error * self.ki + diff_error * self.kd
        self.prev_error = error
        return int(p > 0)

In [3]:
kp = 1
ki = 0
kd = 4

In [4]:
agent = PIDAgent(kp, ki, kd)

env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset(seed=100)
cum_reward = 0

for i in range(100):
    action = agent.get_action(observation)
    observation, reward, terminated, truncated, info = env.step(action)
    cum_reward += reward
    if terminated:
        break

print(f'Reward = {reward:.2f}, n rounds = {i}')
env.close()

Reward = 1.00, n rounds = 99
